In [2]:
#@markdown ### **Installing pip packages**
#@markdown - Diffusion Model: [PyTorch](https://pytorch.org) & [HuggingFace diffusers](https://huggingface.co/docs/diffusers/index)
#@markdown - Dataset Loading: [Zarr](https://zarr.readthedocs.io/en/stable/) & numcodecs
#@markdown - Push-T Env: gym, pygame, pymunk & shapely
!python --version
# !pip3 install torch==1.13.1 torchvision==0.14.1 diffusers==0.18.2 \
# scikit-image==0.19.3 scikit-video==1.1.11 zarr==2.12.0 numcodecs==0.10.2 \
# pygame==2.1.2 pymunk==6.2.1 gym==0.26.2 shapely==1.8.4 \
# &> /dev/null # mute output

Python 3.9.18


In [72]:
#@markdown ### **Imports**
# diffusion policy import
from typing import Tuple, Sequence, Dict, Union, Optional
import numpy as np
import math
import torch
import torch.nn as nn
import collections
import zarr
from diffusers.schedulers.scheduling_ddpm import DDPMScheduler
from diffusers.training_utils import EMAModel
from diffusers.optimization import get_scheduler
from tqdm.auto import tqdm

# env import
import gym
from gym import spaces
import pygame
import pymunk
import pymunk.pygame_util
from pymunk.space_debug_draw_options import SpaceDebugColor
from pymunk.vec2d import Vec2d
import shapely.geometry as sg
import cv2
import skimage.transform as st
from skvideo.io import vwrite
from IPython.display import Video
import gdown
import os

/home/sfchen/miniforge3/envs/robodiff/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


pygame 2.1.2 (SDL 2.0.16, Python 3.9.18)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
#@markdown ### **Environment**
#@markdown Defines a PyMunk-based Push-T environment `PushTEnv`.
#@markdown
#@markdown **Goal**: push the gray T-block into the green area.
#@markdown
#@markdown Adapted from [Implicit Behavior Cloning](https://implicitbc.github.io/)


positive_y_is_up: bool = False
"""Make increasing values of y point upwards.

When True::

    y
    ^
    |      . (3, 3)
    |
    |   . (2, 2)
    |
    +------ > x

When False::

    +------ > x
    |
    |   . (2, 2)
    |
    |      . (3, 3)
    v
    y

"""

def to_pygame(p: Tuple[float, float], surface: pygame.Surface) -> Tuple[int, int]:
    """Convenience method to convert pymunk coordinates to pygame surface
    local coordinates.

    Note that in case positive_y_is_up is False, this function wont actually do
    anything except converting the point to integers.
    """
    if positive_y_is_up:
        return round(p[0]), surface.get_height() - round(p[1])
    else:
        return round(p[0]), round(p[1])


def light_color(color: SpaceDebugColor):
    color = np.minimum(1.2 * np.float32([color.r, color.g, color.b, color.a]), np.float32([255]))
    color = SpaceDebugColor(r=color[0], g=color[1], b=color[2], a=color[3])
    return color

class DrawOptions(pymunk.SpaceDebugDrawOptions):
    def __init__(self, surface: pygame.Surface) -> None:
        """Draw a pymunk.Space on a pygame.Surface object.

        Typical usage::

        >>> import pymunk
        >>> surface = pygame.Surface((10,10))
        >>> space = pymunk.Space()
        >>> options = pymunk.pygame_util.DrawOptions(surface)
        >>> space.debug_draw(options)

        You can control the color of a shape by setting shape.color to the color
        you want it drawn in::

        >>> c = pymunk.Circle(None, 10)
        >>> c.color = pygame.Color("pink")

        See pygame_util.demo.py for a full example

        Since pygame uses a coordiante system where y points down (in contrast
        to many other cases), you either have to make the physics simulation
        with Pymunk also behave in that way, or flip everything when you draw.

        The easiest is probably to just make the simulation behave the same
        way as Pygame does. In that way all coordinates used are in the same
        orientation and easy to reason about::

        >>> space = pymunk.Space()
        >>> space.gravity = (0, -1000)
        >>> body = pymunk.Body()
        >>> body.position = (0, 0) # will be positioned in the top left corner
        >>> space.debug_draw(options)

        To flip the drawing its possible to set the module property
        :py:data:`positive_y_is_up` to True. Then the pygame drawing will flip
        the simulation upside down before drawing::

        >>> positive_y_is_up = True
        >>> body = pymunk.Body()
        >>> body.position = (0, 0)
        >>> # Body will be position in bottom left corner

        :Parameters:
                surface : pygame.Surface
                    Surface that the objects will be drawn on
        """
        self.surface = surface
        super(DrawOptions, self).__init__()

    def draw_circle(
        self,
        pos: Vec2d,
        angle: float,
        radius: float,
        outline_color: SpaceDebugColor,
        fill_color: SpaceDebugColor,
    ) -> None:
        p = to_pygame(pos, self.surface)

        pygame.draw.circle(self.surface, fill_color.as_int(), p, round(radius), 0)
        pygame.draw.circle(self.surface, light_color(fill_color).as_int(), p, round(radius-4), 0)

        circle_edge = pos + Vec2d(radius, 0).rotated(angle)
        p2 = to_pygame(circle_edge, self.surface)
        line_r = 2 if radius > 20 else 1
        # pygame.draw.lines(self.surface, outline_color.as_int(), False, [p, p2], line_r)

    def draw_segment(self, a: Vec2d, b: Vec2d, color: SpaceDebugColor) -> None:
        p1 = to_pygame(a, self.surface)
        p2 = to_pygame(b, self.surface)

        pygame.draw.aalines(self.surface, color.as_int(), False, [p1, p2])

    def draw_fat_segment(
        self,
        a: Tuple[float, float],
        b: Tuple[float, float],
        radius: float,
        outline_color: SpaceDebugColor,
        fill_color: SpaceDebugColor,
    ) -> None:
        p1 = to_pygame(a, self.surface)
        p2 = to_pygame(b, self.surface)

        r = round(max(1, radius * 2))
        pygame.draw.lines(self.surface, fill_color.as_int(), False, [p1, p2], r)
        if r > 2:
            orthog = [abs(p2[1] - p1[1]), abs(p2[0] - p1[0])]
            if orthog[0] == 0 and orthog[1] == 0:
                return
            scale = radius / (orthog[0] * orthog[0] + orthog[1] * orthog[1]) ** 0.5
            orthog[0] = round(orthog[0] * scale)
            orthog[1] = round(orthog[1] * scale)
            points = [
                (p1[0] - orthog[0], p1[1] - orthog[1]),
                (p1[0] + orthog[0], p1[1] + orthog[1]),
                (p2[0] + orthog[0], p2[1] + orthog[1]),
                (p2[0] - orthog[0], p2[1] - orthog[1]),
            ]
            pygame.draw.polygon(self.surface, fill_color.as_int(), points)
            pygame.draw.circle(
                self.surface,
                fill_color.as_int(),
                (round(p1[0]), round(p1[1])),
                round(radius),
            )
            pygame.draw.circle(
                self.surface,
                fill_color.as_int(),
                (round(p2[0]), round(p2[1])),
                round(radius),
            )

    def draw_polygon(
        self,
        verts: Sequence[Tuple[float, float]],
        radius: float,
        outline_color: SpaceDebugColor,
        fill_color: SpaceDebugColor,
    ) -> None:
        ps = [to_pygame(v, self.surface) for v in verts]
        ps += [ps[0]]

        radius = 2
        pygame.draw.polygon(self.surface, light_color(fill_color).as_int(), ps)

        if radius > 0:
            for i in range(len(verts)):
                a = verts[i]
                b = verts[(i + 1) % len(verts)]
                self.draw_fat_segment(a, b, radius, fill_color, fill_color)

    def draw_dot(
        self, size: float, pos: Tuple[float, float], color: SpaceDebugColor
    ) -> None:
        p = to_pygame(pos, self.surface)
        pygame.draw.circle(self.surface, color.as_int(), p, round(size), 0)


def pymunk_to_shapely(body, shapes):
    geoms = list()
    for shape in shapes:
        if isinstance(shape, pymunk.shapes.Poly):
            verts = [body.local_to_world(v) for v in shape.get_vertices()]
            verts += [verts[0]]
            geoms.append(sg.Polygon(verts))
        else:
            raise RuntimeError(f'Unsupported shape type {type(shape)}')
    geom = sg.MultiPolygon(geoms)
    return geom

# env
class PushTEnv(gym.Env):
    metadata = {"render.modes": ["human", "rgb_array"], "video.frames_per_second": 10}
    reward_range = (0., 1.)

    def __init__(self,
            legacy=False,
            block_cog=None, damping=None,
            render_action=True,
            render_size=96,
            reset_to_state=None
        ):
        self._seed = None
        self.seed()
        self.window_size = ws = 512  # The size of the PyGame window
        self.render_size = render_size
        self.sim_hz = 100
        # Local controller params.
        self.k_p, self.k_v = 100, 20    # PD control.z
        self.control_hz = self.metadata['video.frames_per_second']
        # legcay set_state for data compatiblity
        self.legacy = legacy

        # agent_pos, block_pos, block_angle
        self.observation_space = spaces.Box(
            low=np.array([0,0,0,0,0], dtype=np.float64),
            high=np.array([ws,ws,ws,ws,np.pi*2], dtype=np.float64),
            shape=(5,),
            dtype=np.float64
        )

        # positional goal for agent
        self.action_space = spaces.Box(
            low=np.array([0,0], dtype=np.float64),
            high=np.array([ws,ws], dtype=np.float64),
            shape=(2,),
            dtype=np.float64
        )

        self.block_cog = block_cog
        self.damping = damping
        self.render_action = render_action

        """
        If human-rendering is used, `self.window` will be a reference
        to the window that we draw to. `self.clock` will be a clock that is used
        to ensure that the environment is rendered at the correct framerate in
        human-mode. They will remain `None` until human-mode is used for the
        first time.
        """
        self.window = None
        self.clock = None
        self.screen = None

        self.space = None
        self.teleop = None
        self.render_buffer = None
        self.latest_action = None
        self.reset_to_state = reset_to_state

    def reset(self):
        seed = self._seed
        self._setup()
        if self.block_cog is not None:
            self.block.center_of_gravity = self.block_cog
        if self.damping is not None:
            self.space.damping = self.damping

        # use legacy RandomState for compatiblity
        state = self.reset_to_state
        if state is None:
            rs = np.random.RandomState(seed=seed)
            state = np.array([
                rs.randint(50, 450), rs.randint(50, 450),
                rs.randint(100, 400), rs.randint(100, 400),
                rs.randn() * 2 * np.pi - np.pi
                ])
        self._set_state(state)

        obs = self._get_obs()
        info = self._get_info()
        return obs, info

    def step(self, action):
        dt = 1.0 / self.sim_hz
        self.n_contact_points = 0
        n_steps = self.sim_hz // self.control_hz
        if action is not None:
            self.latest_action = action
            for i in range(n_steps):
                # Step PD control.
                # self.agent.velocity = self.k_p * (act - self.agent.position)    # P control works too.
                acceleration = self.k_p * (action - self.agent.position) + self.k_v * (Vec2d(0, 0) - self.agent.velocity)
                self.agent.velocity += acceleration * dt

                # Step physics.
                self.space.step(dt)

        # compute reward
        goal_body = self._get_goal_pose_body(self.goal_pose)
        goal_geom = pymunk_to_shapely(goal_body, self.block.shapes)
        block_geom = pymunk_to_shapely(self.block, self.block.shapes)

        intersection_area = goal_geom.intersection(block_geom).area
        goal_area = goal_geom.area
        coverage = intersection_area / goal_area
        reward = np.clip(coverage / self.success_threshold, 0, 1)
        done = coverage > self.success_threshold
        terminated = done
        truncated = done

        observation = self._get_obs()
        info = self._get_info()

        return observation, reward, terminated, truncated, info

    def render(self, mode):
        return self._render_frame(mode)

    def teleop_agent(self):
        TeleopAgent = collections.namedtuple('TeleopAgent', ['act'])
        def act(obs):
            act = None
            mouse_position = pymunk.pygame_util.from_pygame(Vec2d(*pygame.mouse.get_pos()), self.screen)
            if self.teleop or (mouse_position - self.agent.position).length < 30:
                self.teleop = True
                act = mouse_position
            return act
        return TeleopAgent(act)

    def _get_obs(self):
        obs = np.array(
            tuple(self.agent.position) \
            + tuple(self.block.position) \
            + (self.block.angle % (2 * np.pi),))
        return obs

    def _get_goal_pose_body(self, pose):
        mass = 1
        inertia = pymunk.moment_for_box(mass, (50, 100))
        body = pymunk.Body(mass, inertia)
        # preserving the legacy assignment order for compatibility
        # the order here dosn't matter somehow, maybe because CoM is aligned with body origin
        body.position = pose[:2].tolist()
        body.angle = pose[2]
        return body

    def _get_info(self):
        n_steps = self.sim_hz // self.control_hz
        n_contact_points_per_step = int(np.ceil(self.n_contact_points / n_steps))
        info = {
            'pos_agent': np.array(self.agent.position),
            'vel_agent': np.array(self.agent.velocity),
            'block_pose': np.array(list(self.block.position) + [self.block.angle]),
            'goal_pose': self.goal_pose,
            'n_contacts': n_contact_points_per_step}
        return info

    def _render_frame(self, mode):

        if self.window is None and mode == "human":
            pygame.init()
            pygame.display.init()
            self.window = pygame.display.set_mode((self.window_size, self.window_size))
        if self.clock is None and mode == "human":
            self.clock = pygame.time.Clock()

        canvas = pygame.Surface((self.window_size, self.window_size))
        canvas.fill((255, 255, 255))
        self.screen = canvas

        draw_options = DrawOptions(canvas)

        # Draw goal pose.
        goal_body = self._get_goal_pose_body(self.goal_pose)
        for shape in self.block.shapes:
            goal_points = [pymunk.pygame_util.to_pygame(goal_body.local_to_world(v), draw_options.surface) for v in shape.get_vertices()]
            goal_points += [goal_points[0]]
            pygame.draw.polygon(canvas, self.goal_color, goal_points)

        # Draw agent and block.
        self.space.debug_draw(draw_options)

        if mode == "human":
            # The following line copies our drawings from `canvas` to the visible window
            self.window.blit(canvas, canvas.get_rect())
            pygame.event.pump()
            pygame.display.update()

            # the clock is aleady ticked during in step for "human"


        img = np.transpose(
                np.array(pygame.surfarray.pixels3d(canvas)), axes=(1, 0, 2)
            )
        img = cv2.resize(img, (self.render_size, self.render_size))
        if self.render_action:
            if self.render_action and (self.latest_action is not None):
                action = np.array(self.latest_action)
                coord = (action / 512 * 96).astype(np.int32)
                marker_size = int(8/96*self.render_size)
                thickness = int(1/96*self.render_size)
                cv2.drawMarker(img, coord,
                    color=(255,0,0), markerType=cv2.MARKER_CROSS,
                    markerSize=marker_size, thickness=thickness)
        return img


    def close(self):
        if self.window is not None:
            pygame.display.quit()
            pygame.quit()

    def seed(self, seed=None):
        if seed is None:
            seed = np.random.randint(0,25536)
        self._seed = seed
        self.np_random = np.random.default_rng(seed)

    def _handle_collision(self, arbiter, space, data):
        self.n_contact_points += len(arbiter.contact_point_set.points)

    def _set_state(self, state):
        if isinstance(state, np.ndarray):
            state = state.tolist()
        pos_agent = state[:2]
        pos_block = state[2:4]
        rot_block = state[4]
        self.agent.position = pos_agent
        # setting angle rotates with respect to center of mass
        # therefore will modify the geometric position
        # if not the same as CoM
        # therefore should be modified first.
        if self.legacy:
            # for compatiblity with legacy data
            self.block.position = pos_block
            self.block.angle = rot_block
        else:
            self.block.angle = rot_block
            self.block.position = pos_block

        # Run physics to take effect
        self.space.step(1.0 / self.sim_hz)

    def _set_state_local(self, state_local):
        agent_pos_local = state_local[:2]
        block_pose_local = state_local[2:]
        tf_img_obj = st.AffineTransform(
            translation=self.goal_pose[:2],
            rotation=self.goal_pose[2])
        tf_obj_new = st.AffineTransform(
            translation=block_pose_local[:2],
            rotation=block_pose_local[2]
        )
        tf_img_new = st.AffineTransform(
            matrix=tf_img_obj.params @ tf_obj_new.params
        )
        agent_pos_new = tf_img_new(agent_pos_local)
        new_state = np.array(
            list(agent_pos_new[0]) + list(tf_img_new.translation) \
                + [tf_img_new.rotation])
        self._set_state(new_state)
        return new_state

    def _setup(self):
        self.space = pymunk.Space()
        self.space.gravity = 0, 0
        self.space.damping = 0
        self.teleop = False
        self.render_buffer = list()

        # Add walls.
        walls = [
            self._add_segment((5, 506), (5, 5), 2),
            self._add_segment((5, 5), (506, 5), 2),
            self._add_segment((506, 5), (506, 506), 2),
            self._add_segment((5, 506), (506, 506), 2)
        ]
        self.space.add(*walls)

        # Add agent, block, and goal zone.
        self.agent = self.add_circle((256, 400), 15)
        self.block = self.add_tee((256, 300), 0)
        self.goal_color = pygame.Color('LightGreen')
        self.goal_pose = np.array([256,256,np.pi/4])  # x, y, theta (in radians)

        # Add collision handeling
        self.collision_handeler = self.space.add_collision_handler(0, 0)
        self.collision_handeler.post_solve = self._handle_collision
        self.n_contact_points = 0

        self.max_score = 50 * 100
        self.success_threshold = 0.95    # 95% coverage.

    def _add_segment(self, a, b, radius):
        shape = pymunk.Segment(self.space.static_body, a, b, radius)
        shape.color = pygame.Color('LightGray')    # https://htmlcolorcodes.com/color-names
        return shape

    def add_circle(self, position, radius):
        body = pymunk.Body(body_type=pymunk.Body.KINEMATIC)
        body.position = position
        body.friction = 1
        shape = pymunk.Circle(body, radius)
        shape.color = pygame.Color('RoyalBlue')
        self.space.add(body, shape)
        return body

    def add_box(self, position, height, width):
        mass = 1
        inertia = pymunk.moment_for_box(mass, (height, width))
        body = pymunk.Body(mass, inertia)
        body.position = position
        shape = pymunk.Poly.create_box(body, (height, width))
        shape.color = pygame.Color('LightSlateGray')
        self.space.add(body, shape)
        return body

    def add_tee(self, position, angle, scale=30, color='LightSlateGray', mask=pymunk.ShapeFilter.ALL_MASKS()):
        mass = 1
        length = 4
        vertices1 = [(-length*scale/2, scale),
                                 ( length*scale/2, scale),
                                 ( length*scale/2, 0),
                                 (-length*scale/2, 0)]
        inertia1 = pymunk.moment_for_poly(mass, vertices=vertices1)
        vertices2 = [(-scale/2, scale),
                                 (-scale/2, length*scale),
                                 ( scale/2, length*scale),
                                 ( scale/2, scale)]
        inertia2 = pymunk.moment_for_poly(mass, vertices=vertices1)
        body = pymunk.Body(mass, inertia1 + inertia2)
        shape1 = pymunk.Poly(body, vertices1)
        shape2 = pymunk.Poly(body, vertices2)
        shape1.color = pygame.Color(color)
        shape2.color = pygame.Color(color)
        shape1.filter = pymunk.ShapeFilter(mask=mask)
        shape2.filter = pymunk.ShapeFilter(mask=mask)
        body.center_of_gravity = (shape1.center_of_gravity + shape2.center_of_gravity) / 2
        body.position = position
        body.angle = angle
        body.friction = 1
        self.space.add(body, shape1, shape2)
        return body


In [73]:
from diffusion_policy.env.block_pushing.block_pushing import BlockPush, BlockPushNormalized
from diffusion_policy.env.block_pushing.block_pushing_discontinuous import BlockPushDiscontinuous
from diffusion_policy.env.block_pushing.block_pushing_multimodal import BlockPushMultimodal
#@markdown ### **Env Demo**
#@markdown Standard Gym Env (0.21.0 API)

# 0. create env object
# env = PushTEnv()
# env = BlockPush()
# env = BlockPushNormalized()
# env = BlockPushDiscontinuous()
env = BlockPushMultimodal()

# 1. seed env for initial state.
# Seed 0-200 are used for the demonstration dataset.
env.seed(1000)

# 2. must reset before use
# obs, DEFAULT_IGNORE_PATTERNS = env.reset()
obs = env.reset()

# 3. 2D positional action space [0,512]
action = env.action_space.sample()

# 4. Standard gym step method
# obs, reward, terminated, truncated, info = env.step(action)
obs, reward, terminated, info = env.step(action)

# prints and explains each dimension of the observation and action vectors
with np.printoptions(precision=4, suppress=True, threshold=5):
    # print("Obs: ", repr(obs))
    # print("Obs:        [agent_x,  agent_y,  block_x,  block_y,    block_angle]")
    print('Obs:')
    for k,v in obs.items():
        print(f"\t{k}: {v}")
    print("Action: ", repr(action))
    # print("Action:   [target_agent_x, target_agent_y]")

/home/sfchen/miniforge3/envs/robodiff/lib/python3.9/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


argv[0]=
Obs:
	block_translation: (0.43071725041927583, -0.3154978396411959)
	block_orientation: [1.1064]
	block2_translation: (0.31391576244968816, -0.08439863792609263)
	block2_orientation: [1.1014]
	effector_translation: [ 0.3268 -0.487 ]
	effector_target_translation: [ 0.329  -0.4977]
	target_translation: [0.2774 0.2024]
	target_orientation: [3.1367]
	target2_translation: [0.5232 0.1961]
	target2_orientation: [0.0148]
Action:  array([ 0.029 , -0.0977], dtype=float32)


In [6]:
#@markdown ### **Dataset**
#@markdown
#@markdown Defines `PushTStateDataset` and helper functions
#@markdown
#@markdown The dataset class
#@markdown - Load data (obs, action) from a zarr storage
#@markdown - Normalizes each dimension of obs and action to [-1,1]
#@markdown - Returns
#@markdown  - All possible segments with length `pred_horizon`
#@markdown  - Pads the beginning and the end of each episode with repetition
#@markdown  - key `obs`: shape (obs_horizon, obs_dim)
#@markdown  - key `action`: shape (pred_horizon, action_dim)

def create_sample_indices(
        episode_ends:np.ndarray, sequence_length:int,
        pad_before: int=0, pad_after: int=0):
    indices = list()
    for i in range(len(episode_ends)):
        start_idx = 0
        if i > 0:
            start_idx = episode_ends[i-1]
        end_idx = episode_ends[i]
        episode_length = end_idx - start_idx

        min_start = -pad_before
        max_start = episode_length - sequence_length + pad_after

        # range stops one idx before end
        for idx in range(min_start, max_start+1):
            buffer_start_idx = max(idx, 0) + start_idx
            buffer_end_idx = min(idx+sequence_length, episode_length) + start_idx
            start_offset = buffer_start_idx - (idx+start_idx)
            end_offset = (idx+sequence_length+start_idx) - buffer_end_idx
            sample_start_idx = 0 + start_offset
            sample_end_idx = sequence_length - end_offset
            indices.append([
                buffer_start_idx, buffer_end_idx,
                sample_start_idx, sample_end_idx])
    indices = np.array(indices)
    return indices


def sample_sequence(train_data, sequence_length,
                    buffer_start_idx, buffer_end_idx,
                    sample_start_idx, sample_end_idx):
    result = dict()
    for key, input_arr in train_data.items():
        sample = input_arr[buffer_start_idx:buffer_end_idx]
        data = sample
        if (sample_start_idx > 0) or (sample_end_idx < sequence_length):
            data = np.zeros(
                shape=(sequence_length,) + input_arr.shape[1:],
                dtype=input_arr.dtype)
            if sample_start_idx > 0:
                data[:sample_start_idx] = sample[0]
            if sample_end_idx < sequence_length:
                data[sample_end_idx:] = sample[-1]
            data[sample_start_idx:sample_end_idx] = sample
        result[key] = data
    return result

# normalize data
def get_data_stats(data):
    data = data.reshape(-1,data.shape[-1])
    stats = {
        'min': np.min(data, axis=0),
        'max': np.max(data, axis=0)
    }
    return stats

def normalize_data(data, stats):
    # nomalize to [0,1]
    ndata = (data - stats['min']) / (stats['max'] - stats['min'])
    # normalize to [-1, 1]
    ndata = ndata * 2 - 1
    return ndata

def unnormalize_data(ndata, stats):
    ndata = (ndata + 1) / 2
    data = ndata * (stats['max'] - stats['min']) + stats['min']
    return data

# dataset
class PushTStateDataset(torch.utils.data.Dataset):
    def __init__(self, dataset_path,
                 pred_horizon, obs_horizon, action_horizon):

        # read from zarr dataset
        dataset_root = zarr.open(dataset_path, 'r')
        # All demonstration episodes are concatinated in the first dimension N
        train_data = {
            # (N, action_dim)
            'action': dataset_root['data']['action'][:],
            # (N, obs_dim)
            'obs': dataset_root['data']['state'][:]
        }
        # Marks one-past the last index for each episode
        episode_ends = dataset_root['meta']['episode_ends'][:]

        # compute start and end of each state-action sequence
        # also handles padding
        indices = create_sample_indices(
            episode_ends=episode_ends,
            sequence_length=pred_horizon,
            # add padding such that each timestep in the dataset are seen
            pad_before=obs_horizon-1,
            pad_after=action_horizon-1)

        # compute statistics and normalized data to [-1,1]
        stats = dict()
        normalized_train_data = dict()
        for key, data in train_data.items():
            stats[key] = get_data_stats(data)
            normalized_train_data[key] = normalize_data(data, stats[key])

        self.indices = indices
        self.stats = stats
        self.normalized_train_data = normalized_train_data
        self.pred_horizon = pred_horizon
        self.action_horizon = action_horizon
        self.obs_horizon = obs_horizon

    def __len__(self):
        # all possible segments of the dataset
        return len(self.indices)

    def __getitem__(self, idx):
        # get the start/end indices for this datapoint
        buffer_start_idx, buffer_end_idx, \
            sample_start_idx, sample_end_idx = self.indices[idx]

        # get nomralized data using these indices
        nsample = sample_sequence(
            train_data=self.normalized_train_data,
            sequence_length=self.pred_horizon,
            buffer_start_idx=buffer_start_idx,
            buffer_end_idx=buffer_end_idx,
            sample_start_idx=sample_start_idx,
            sample_end_idx=sample_end_idx
        )

        # discard unused observations
        nsample['obs'] = nsample['obs'][:self.obs_horizon,:]
        return nsample


In [33]:
#@markdown ### **Dataset Demo**
from diffusion_policy.dataset.blockpush_lowdim_dataset import BlockPushLowdimDataset

# download demonstration data from Google Drive
# dataset_path = "pusht_cchi_v7_replay.zarr.zip"
dataset_path = "data/training/block_pushing/multimodal_push_seed_abs.zarr"
# if not os.path.isfile(dataset_path):
    # id = "1KY1InLurpMvJDRb14L9NlXT_fEsCvVUq&confirm=t"
    # gdown.download(id=id, output=dataset_path, quiet=False)

# parameters
pred_horizon = 16
obs_horizon = 2
action_horizon = 8
#|o|o|                             observations: 2
#| |a|a|a|a|a|a|a|a|               actions executed: 8
#|p|p|p|p|p|p|p|p|p|p|p|p|p|p|p|p| actions predicted: 16

# create dataset from file
# dataset = PushTStateDataset(
    # dataset_path=dataset_path,
    # pred_horizon=pred_horizon,
    # obs_horizon=obs_horizon,
    # action_horizon=action_horizon
# )
dataset = BlockPushLowdimDataset(
    zarr_path=dataset_path,
    horizon=pred_horizon,
    pad_before=obs_horizon-1,
    pad_after=action_horizon-1
)
# save training data statistics (min, max) for each dim
# stats = dataset.stats

# create dataloader
dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=256,
    num_workers=1,
    shuffle=True,
    # accelerate cpu-gpu transfer
    pin_memory=True,
    # don't kill worker process afte each epoch
    persistent_workers=True
)

# visualize data in batch
batch = next(iter(dataloader))
print("batch['obs'].shape:", batch['obs'].shape)
print("batch['action'].shape", batch['action'].shape)

batch['obs'].shape: torch.Size([256, 16, 16])
batch['action'].shape torch.Size([256, 16, 2])


In [60]:
import os
import numpy as np
import torch
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from diffusion_policy.env.block_pushing.block_pushing_multimodal import BlockPushMultimodal
from diffusion_policy.dataset.blockpush_lowdim_dataset import BlockPushLowdimDataset
from matplotlib.patches import Rectangle, Circle

# Parameters
dataset_path = "data/training/block_pushing/multimodal_push_seed_abs.zarr"
pred_horizon = 128
obs_horizon = 2
action_horizon = 8

# Initialize Dataset and Dataloader
dataset = BlockPushLowdimDataset(
    zarr_path=dataset_path,
    horizon=pred_horizon,
    pad_before=obs_horizon - 1,
    pad_after=action_horizon - 1
)

dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=1,  # Use batch_size=1 for visualization purposes
    num_workers=1,
    shuffle=True,
    pin_memory=True,
    persistent_workers=True
)

# Load a batch from the dataset
for i in range(50):
    batch = next(iter(dataloader))
    obs_batch = batch['obs']  # Shape: [batch_size, time, dim]
    action_batch = batch['action']  # Shape: [batch_size, time, dim]

# Create environment
env = BlockPushMultimodal()

print("Trajectory visualization saved as block_push_trajectory.mp4")


/home/sfchen/miniforge3/envs/robodiff/lib/python3.9/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


argv[0]=
Trajectory visualization saved as block_push_trajectory.mp4


In [58]:
import os
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from diffusion_policy.env.block_pushing.block_pushing_multimodal import BlockPushMultimodal
from diffusion_policy.dataset.blockpush_lowdim_dataset import BlockPushLowdimDataset
from matplotlib.patches import Rectangle, Circle
from tqdm import tqdm
from torch.optim.swa_utils import AveragedModel as EMAModel

# Parameters
dataset_path = "data/training/block_pushing/multimodal_push_seed_abs.zarr"
pred_horizon = 128
obs_horizon = 2
action_horizon = 8

# Initialize Dataset and Dataloader
dataset = BlockPushLowdimDataset(
    zarr_path=dataset_path,
    horizon=pred_horizon,
    pad_before=obs_horizon - 1,
    pad_after=action_horizon - 1
)

# Function to split trajectories into two parts
class SplitBlockPushLowdimDataset(BlockPushLowdimDataset):
    def __init__(self, original_dataset, dataset_path, split="first"):
        super().__init__(
            zarr_path=dataset_path,
            horizon=original_dataset.horizon,
            pad_before=original_dataset.pad_before,
            pad_after=original_dataset.pad_after,
            obs_key=original_dataset.obs_key,
            action_key=original_dataset.action_key,
            obs_eef_target=original_dataset.obs_eef_target,
            use_manual_normalizer=original_dataset.use_manual_normalizer,
            seed=42,
            val_ratio=0.0
        )
        self.split = split

    def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:
        sample = self.sampler.sample_sequence(idx)
        data = self._sample_to_data(sample)

        # Split the trajectory into two halves
        split_index = data['obs'].shape[0] // 2
        if self.split == "first":
            obs = data['obs'][:split_index]
            action = data['action'][:split_index]
        elif self.split == "second":
            obs = data['obs'][split_index:]
            action = data['action'][split_index:]
        else:
            raise ValueError(f"Invalid split: {self.split}")

        torch_data = {
            'obs': torch.from_numpy(obs),
            'action': torch.from_numpy(action)
        }
        return torch_data

# Create split datasets
dataset_first_half = SplitBlockPushLowdimDataset(dataset, dataset_path=dataset_path, split="first")
dataset_second_half = SplitBlockPushLowdimDataset(dataset, dataset_path=dataset_path, split="second")

# Function to filter out trajectories where two blocks move together
# This filter will exclude any trajectory where the blocks are seen moving simultaneously (i.e., interacting in a way that suggests they are both affected by the effector)
def filter_no_two_blocks_moving_together(data):
    obs = data['obs']
    threshold_distance = 0.05  # Minimum distance to consider as "too close"
    close_count = 0  # Counter for how many consecutive timesteps the blocks are close

    for t in range(1, len(obs)):
        block1_translation = obs[t][:2]
        block2_translation = obs[t][3:5]
        distance_between_blocks = np.linalg.norm(block1_translation - block2_translation)

        # Check if the blocks are close together
        if distance_between_blocks < threshold_distance:
            close_count += 1
            # If blocks are close for at least 10 consecutive timesteps, filter out this trajectory
            if close_count >= 10:
                return False
        else:
            close_count = 0

    return True


# Function to filter dataset to create dataset1 (pushing block1 to region1 or region2)
def filter_block1_to_region1_or_region2(data):
    obs = data['obs']
    block2_translation_initial = obs[0][3:5]
    block2_translation_final = obs[-1][3:5]
    block2_orientation_initial = obs[0][5]
    block2_orientation_final = obs[-1][5]
    # Ensure block2 does not change its position or orientation throughout timesteps
    if (not np.allclose(block2_translation_initial, block2_translation_final, atol=1e-3) or
            not np.isclose(block2_orientation_initial, block2_orientation_final, atol=1e-3)):
        return False
    return True

# Function to filter dataset
class FilteredBlockPushLowdimDataset(BlockPushLowdimDataset):
    def __init__(self, original_dataset, filter_fn):
        self.original_dataset = original_dataset
        self.filter_fn = filter_fn
        self.filtered_indices = [i for i in range(len(original_dataset)) if filter_fn(original_dataset[i])]

    def __len__(self):
        return len(self.filtered_indices)

    def __getitem__(self, idx):
        return self.original_dataset[self.filtered_indices[idx]]

# Create filtered dataset1 using both first and second half datasets
filtered_dataset1_first_half = FilteredBlockPushLowdimDataset(dataset_first_half, filter_block1_to_region1_or_region2)
filtered_dataset1_second_half = FilteredBlockPushLowdimDataset(dataset_second_half, filter_block1_to_region1_or_region2)

# Apply filter to remove trajectories where two blocks move together for dataset1
filtered_dataset1_first_half = FilteredBlockPushLowdimDataset(filtered_dataset1_first_half, filter_no_two_blocks_moving_together)
filtered_dataset1_second_half = FilteredBlockPushLowdimDataset(filtered_dataset1_second_half, filter_no_two_blocks_moving_together)

# Combine filtered datasets to create dataset1
dataset1 = torch.utils.data.ConcatDataset([filtered_dataset1_first_half, filtered_dataset1_second_half])

print("Filtered dataset1 created containing trajectories involving pushing block1 to region1 or region2 and excluding cases where two blocks move together.")

# Function to filter dataset to create dataset2 (pushing block1 or block2 to region1)
def filter_block1_or_block2_to_region1(data):
    obs = data['obs']
    # Extract region centers directly from the observation data
    region1_center = obs[0][10:12]  # Target translation (region 1)
    region2_center = obs[0][13:15]  # Target2 translation (region 2)
    region_radius = 0.05  # Radius to define the vicinity of the region

    # Check if no block is near region 1 at timestep 1
    block1_translation_initial = obs[0][:2]
    block2_translation_initial = obs[0][3:5]
    if (np.linalg.norm(block1_translation_initial - region1_center) < region_radius or
            np.linalg.norm(block2_translation_initial - region1_center) < region_radius):
        return False

    # Check if either block1 or block2 is near region 1 at the last timestep
    block1_translation_final = obs[-1][:2]
    block2_translation_final = obs[-1][3:5]
    if (np.linalg.norm(block1_translation_final - region1_center) > region_radius and
            np.linalg.norm(block2_translation_final - region1_center) > region_radius):
        return False
    return True

# Create filtered dataset2 using both first and second half datasets
filtered_dataset2_first_half = FilteredBlockPushLowdimDataset(dataset_first_half, filter_block1_or_block2_to_region1)
filtered_dataset2_second_half = FilteredBlockPushLowdimDataset(dataset_second_half, filter_block1_or_block2_to_region1)

# Apply filter to remove trajectories where two blocks move together for dataset2
filtered_dataset2_first_half = FilteredBlockPushLowdimDataset(filtered_dataset2_first_half, filter_no_two_blocks_moving_together)
filtered_dataset2_second_half = FilteredBlockPushLowdimDataset(filtered_dataset2_second_half, filter_no_two_blocks_moving_together)

# Combine filtered datasets to create dataset2
dataset2 = torch.utils.data.ConcatDataset([filtered_dataset2_first_half, filtered_dataset2_second_half])
print("Filtered dataset2 created containing trajectories involving pushing block1 or block2 to region1.")



Filtered dataset1 created containing trajectories involving pushing block1 to region1 or region2 and excluding cases where two blocks move together.
Filtered dataset2 created containing trajectories involving pushing block1 or block2 to region1.


In [62]:
# Visualization function for dataset1 and dataset2
def visualize_trajectory(obs, actions, save_path="trajectory.mp4"):
    fig, ax = plt.subplots()
    ax.set_xlim(-0.5, 1.5)
    ax.set_ylim(-0.5, 1.5)
    agent_patch = Rectangle((0, 0), 0.02, 0.02, fc='blue', label='Effector')
    block_patch = Rectangle((0, 0), 0.03, 0.03, fc='red', label='Block')
    block2_patch = Rectangle((0, 0), 0.03, 0.03, fc='green', label='Block 2')
    target_patch = Circle((0, 0), 0.02, fc='orange', label='Target')
    target2_patch = Circle((0, 0), 0.02, fc='purple', label='Target 2')
    region1_circle = Circle((0, 0), 0.05, fill=False, edgecolor='orange', linestyle='--', label='Region 1 Boundary')
    region2_circle = Circle((0, 0), 0.05, fill=False, edgecolor='purple', linestyle='--', label='Region 2 Boundary')
    ax.add_patch(agent_patch)
    ax.add_patch(block_patch)
    ax.add_patch(block2_patch)
    ax.add_patch(target_patch)
    ax.add_patch(target2_patch)
    ax.add_patch(region1_circle)
    ax.add_patch(region2_circle)
    plt.legend()

    def update(frame):
        block_translation = obs[frame][:2]
        block_orientation = obs[frame][2]
        block2_translation = obs[frame][3:5]
        block2_orientation = obs[frame][5]
        effector_translation = obs[frame][6:8]
        effector_target_translation = obs[frame][8:10]
        target_translation = obs[frame][10:12]
        target_orientation = obs[frame][12]
        target2_translation = obs[frame][13:15]
        target2_orientation = obs[frame][15]
        target_agent_x, target_agent_y = actions[frame][:2]
        agent_patch.set_xy((effector_translation[0], effector_translation[1]))
        block_patch.set_xy((block_translation[0], block_translation[1]))
        block2_patch.set_xy((block2_translation[0], block2_translation[1]))
        target_patch.set_center((target_translation[0], target_translation[1]))
        target2_patch.set_center((target2_translation[0], target2_translation[1]))
        region1_circle.set_center((target_translation[0], target_translation[1]))
        region2_circle.set_center((target2_translation[0], target2_translation[1]))
        ax.set_title(f"Frame: {frame}, Target Agent: ({target_agent_x:.2f}, {target_agent_y:.2f}" + "\n"
                     f"Frame: {frame}, Current Agent: ({effector_translation[0]:.2f}, {effector_target_translation[1]:.2f})")
        return agent_patch, block_patch, block2_patch, target_patch, target2_patch, region1_circle, region2_circle

    ani = animation.FuncAnimation(fig, update, frames=len(obs), blit=False, repeat=False)
    ani.save(save_path, writer='ffmpeg', fps=5)
    plt.close()


In [63]:
import random

def visualize_dataset(dataset, save_dir, dataset_name):
    os.makedirs(save_dir, exist_ok=True)
    indices = random.sample(range(len(dataset)), min(len(dataset), 3))  # Pick 10 random trajectories for visualization
    for i in indices:  # Visualize up to 5 trajectories for verification
        data = dataset[i]
        obs = data['obs'].numpy()
        actions = data['action'].numpy()
        save_path = os.path.join(save_dir, f"{dataset_name}_trajectory_{i}.mp4")
        visualize_trajectory(obs, actions, save_path)
        print(f"{dataset_name} trajectory {i} visualization saved as {save_path}")

# Visualize and save trajectories from dataset1 and dataset2
visualize_dataset(dataset1, save_dir="visualizations/dataset1", dataset_name="dataset1")
visualize_dataset(dataset2, save_dir="visualizations/dataset2", dataset_name="dataset2")


2024-11-26 19:11:07,494 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
2024-11-26 19:11:07,496 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 640x480 -pix_fmt rgba -r 5 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -y visualizations/dataset1/dataset1_trajectory_640.mp4
2024-11-26 19:11:20,199 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
2024-11-26 19:11:20,201 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 640x480 -pix_fmt rgba -r 5 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -y visualizations/dataset1/dataset1_trajectory_688.mp4


dataset1 trajectory 640 visualization saved as visualizations/dataset1/dataset1_trajectory_640.mp4


2024-11-26 19:11:34,781 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
2024-11-26 19:11:34,783 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 640x480 -pix_fmt rgba -r 5 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -y visualizations/dataset1/dataset1_trajectory_491.mp4


dataset1 trajectory 688 visualization saved as visualizations/dataset1/dataset1_trajectory_688.mp4


2024-11-26 19:11:46,969 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
2024-11-26 19:11:46,971 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 640x480 -pix_fmt rgba -r 5 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -y visualizations/dataset2/dataset2_trajectory_34.mp4


dataset1 trajectory 491 visualization saved as visualizations/dataset1/dataset1_trajectory_491.mp4


2024-11-26 19:11:57,356 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
2024-11-26 19:11:57,358 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 640x480 -pix_fmt rgba -r 5 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -y visualizations/dataset2/dataset2_trajectory_270.mp4


dataset2 trajectory 34 visualization saved as visualizations/dataset2/dataset2_trajectory_34.mp4


2024-11-26 19:12:07,682 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
2024-11-26 19:12:07,684 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 640x480 -pix_fmt rgba -r 5 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -y visualizations/dataset2/dataset2_trajectory_1304.mp4


dataset2 trajectory 270 visualization saved as visualizations/dataset2/dataset2_trajectory_270.mp4
dataset2 trajectory 1304 visualization saved as visualizations/dataset2/dataset2_trajectory_1304.mp4


In [71]:
def categorize_trajectories(dataset, dataset_name):
    categories = {
        "pushing_block1_to_region1": 0,
        "pushing_block2_to_region1": 0,
        "pushing_block1_to_region2": 0
    }
    pushing_block1_to_region2_indices = []  # To record indices of incorrect cases

    vs = [0,0,0]
    for idx, data in enumerate(dataset):
        obs = data['obs'].numpy()

        region1_center = obs[0][10:12]
        region2_center = obs[0][13:15]

        block1_translation_initial = obs[0][:2]
        block2_translation_initial = obs[0][3:5]
        block1_translation_final = obs[-1][:2]
        block2_translation_final = obs[-1][3:5]
        region_radius = 0.05

        # Check conditions for pushing block1 to region1
        if (np.linalg.norm(block1_translation_initial - region1_center) > region_radius and
                np.linalg.norm(block1_translation_final - region1_center) < region_radius):
            categories["pushing_block1_to_region1"] += 1

            if vs[0] == 0:
                actions = data['action'].numpy()
                save_path = os.path.join(f'./visualizations/{dataset_name}', f"{dataset_name}_trajectory_{idx}_block1_region1.mp4")
                visualize_trajectory(obs, actions, save_path)
                vs[0] = 1

        # Check conditions for pushing block2 to region1
        elif (np.linalg.norm(block2_translation_initial - region1_center) > region_radius and
                np.linalg.norm(block2_translation_final - region1_center) < region_radius):
            categories["pushing_block2_to_region1"] += 1

            if vs[1] == 0:
                actions = data['action'].numpy()
                save_path = os.path.join('./visualizations', f"{dataset_name}_trajectory_{idx}_block2_region1.mp4")
                visualize_trajectory(obs, actions, save_path)
                vs[1] = 1

        # Check conditions for pushing block1 to region2
        elif (np.linalg.norm(block1_translation_initial - region2_center) > region_radius and
                np.linalg.norm(block1_translation_final - region2_center) < region_radius):
            categories["pushing_block1_to_region2"] += 1

            if vs[2] == 0:
                actions = data['action'].numpy()
                save_path = os.path.join(f'./visualizations/{dataset_name}', f"{dataset_name}_trajectory_{idx}_block1_region2.mp4")
                visualize_trajectory(obs, actions, save_path)
                vs[2] = 1

    print("Trajectory categorization statistics:")
    for category, count in categories.items():
        print(f"{category}: {count}")

    # Print the indices of the incorrect cases for dataset2
    if pushing_block1_to_region2_indices:
        print("Indices of trajectories in dataset2 that incorrectly involve pushing block1 to region2:")
        print(pushing_block1_to_region2_indices)
        obs = data['obs'].numpy()

   

# Categorize trajectories in dataset1 and dataset2
categorize_trajectories(dataset1, 'dataset1')
categorize_trajectories(dataset2, 'dataset2')

2024-11-27 11:23:42,037 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
2024-11-27 11:23:42,039 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 640x480 -pix_fmt rgba -r 5 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -y ./visualizations/dataset1/dataset1_trajectory_0_block1_region2.mp4
2024-11-27 11:23:57,921 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
2024-11-27 11:23:57,924 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 640x480 -pix_fmt rgba -r 5 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -y ./visualizations/dataset1/dataset1_trajectory_7_block1_region1.mp4
2024-11-27 11:24:12,710 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
2024-11-27 11:24:12,713 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 640x480 -pix_fmt rgba -r 5 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -y ./visual

Trajectory categorization statistics:
pushing_block1_to_region1: 621
pushing_block2_to_region1: 0
pushing_block1_to_region2: 674


2024-11-27 11:24:26,192 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
2024-11-27 11:24:26,193 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 640x480 -pix_fmt rgba -r 5 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -y ./visualizations/dataset2_trajectory_2_block2_region1.mp4


Trajectory categorization statistics:
pushing_block1_to_region1: 723
pushing_block2_to_region1: 845
pushing_block1_to_region2: 0


In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from diffusion_policy.env.block_pushing.block_pushing_multimodal import BlockPushMultimodal
from diffusion_policy.dataset.blockpush_lowdim_dataset import BlockPushLowdimDataset
from matplotlib.patches import Rectangle, Circle
from tqdm import tqdm
from torch.optim.swa_utils import AveragedModel as EMAModel
from diffusion_policy.common.sampler import SequenceSampler, get_val_mask
from typing import Dict

# Parameters
dataset_path = "data/training/block_pushing/multimodal_push_seed_abs.zarr"
pred_horizon = 128
obs_horizon = 2
action_horizon = 8

# Initialize Dataset and Dataloader
dataset = BlockPushLowdimDataset(
    zarr_path=dataset_path,
    horizon=pred_horizon,
    pad_before=obs_horizon - 1,
    pad_after=action_horizon - 1
)

# Function to split trajectories into two parts
class SplitBlockPushLowdimDataset(BlockPushLowdimDataset):
    def __init__(self, original_dataset, split="first"):
        # Directly copy the attributes from the original dataset without calling super().__init__()
        self.replay_buffer = original_dataset.replay_buffer  # Reuse the original replay buffer
        self.obs_key = original_dataset.obs_key
        self.action_key = original_dataset.action_key
        self.obs_eef_target = original_dataset.obs_eef_target
        self.use_manual_normalizer = original_dataset.use_manual_normalizer
        self.train_mask = original_dataset.train_mask

        # Assign other parameters from original dataset
        self.horizon = original_dataset.horizon
        self.pad_before = original_dataset.pad_before
        self.pad_after = original_dataset.pad_after
        self.split = split

        # Initialize the sampler to be used for splitting
        self.sampler = original_dataset.sampler

    def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:
        # Use the sampler from original dataset to get a sample
        sample = self.sampler.sample_sequence(idx)
        data = self._sample_to_data(sample)

        # Split the trajectory into two halves
        split_index = data['obs'].shape[0] // 2
        if self.split == "first":
            obs = data['obs'][:split_index]
            action = data['action'][:split_index]
        elif self.split == "second":
            obs = data['obs'][split_index:]
            action = data['action'][split_index:]
        else:
            raise ValueError(f"Invalid split: {self.split}")

        # Convert to torch Tensors
        torch_data = {
            'obs': torch.from_numpy(obs),
            'action': torch.from_numpy(action)
        }
        return torch_data

    def __len__(self) -> int:
        return len(self.sampler)


# Create split datasets
dataset_first_half = SplitBlockPushLowdimDataset(original_dataset=dataset, split="first")
dataset_second_half = SplitBlockPushLowdimDataset(original_dataset=dataset, split="second")

# Function to filter dataset
class FilteredBlockPushLowdimDataset(BlockPushLowdimDataset):
    def __init__(self, original_dataset, filter_fn):
        # Apply filter to find relevant indices
        self.filter_fn = filter_fn
        self.filtered_indices = [i for i in range(len(original_dataset)) if filter_fn(original_dataset[i])]

        # Assign necessary attributes from the original dataset
        self.replay_buffer = original_dataset.replay_buffer  # Inherit replay buffer
        self.obs_key = original_dataset.obs_key
        self.action_key = original_dataset.action_key
        self.obs_eef_target = original_dataset.obs_eef_target
        self.use_manual_normalizer = original_dataset.use_manual_normalizer
        self.train_mask = original_dataset.train_mask

        # Use the filtered indices and reset the sampler accordingly
        self.reset_sampler(
            horizon=original_dataset.horizon,
            pad_before=original_dataset.pad_before,
            pad_after=original_dataset.pad_after
        )

    def __len__(self):
        return len(self.filtered_indices)

    def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:
        # Access the filtered index to retrieve the sample
        filtered_idx = self.filtered_indices[idx]
        sample = self.sampler.sample_sequence(filtered_idx)
        data = self._sample_to_data(sample)

        # Convert to torch Tensors
        torch_data = {
            'obs': torch.from_numpy(data['obs']),
            'action': torch.from_numpy(data['action'])
        }
        return torch_data

    def reset_sampler(self, horizon, pad_before, pad_after):
        """
        Resets the SequenceSampler with new sequence parameters.
        """
        self.horizon = horizon
        self.pad_before = pad_before
        self.pad_after = pad_after

        # Create a new SequenceSampler with updated sequence parameters
        # Use the same replay buffer but filter the episode_mask to only include filtered_indices
        filtered_episode_mask = np.zeros(self.replay_buffer.n_episodes, dtype=bool)
        filtered_episode_mask[self.filtered_indices] = True

        self.sampler = SequenceSampler(
            replay_buffer=self.replay_buffer,
            sequence_length=horizon,
            pad_before=pad_before,
            pad_after=pad_after,
            episode_mask=filtered_episode_mask
        )
        print(f"Sampler reset with horizon={horizon}, pad_before={pad_before}, pad_after={pad_after}")

# Filtering function to remove trajectories where two blocks move together
def filter_no_two_blocks_moving_together(data):
    obs = data['obs']
    threshold_distance = 0.05  # Minimum distance to consider as "too close"
    close_count = 0  # Counter for how many consecutive timesteps the blocks are close

    for t in range(1, len(obs)):
        block1_translation = obs[t][:2]
        block2_translation = obs[t][3:5]
        distance_between_blocks = np.linalg.norm(block1_translation - block2_translation)

        # Check if the blocks are close together
        if distance_between_blocks < threshold_distance:
            close_count += 1
            # If blocks are close for at least 10 consecutive timesteps, filter out this trajectory
            if close_count >= 10:
                return False
        else:
            close_count = 0

    return True

# Create filtered dataset1 using both first and second half datasets
filtered_dataset1_first_half = FilteredBlockPushLowdimDataset(dataset_first_half, filter_block1_to_region1_or_region2)
filtered_dataset1_second_half = FilteredBlockPushLowdimDataset(dataset_second_half, filter_block1_to_region1_or_region2)

# Apply filter to remove trajectories where two blocks move together for dataset1
filtered_dataset1_first_half = FilteredBlockPushLowdimDataset(filtered_dataset1_first_half, filter_no_two_blocks_moving_together)
filtered_dataset1_second_half = FilteredBlockPushLowdimDataset(filtered_dataset1_second_half, filter_no_two_blocks_moving_together)

# Combine filtered datasets to create dataset1
dataset1 = torch.utils.data.ConcatDataset([filtered_dataset1_first_half, filtered_dataset1_second_half])
print("Filtered dataset1 created containing trajectories involving pushing block1 to region1 or region2 and excluding cases where two blocks move together.")

# Create filtered dataset2 using both first and second half datasets
filtered_dataset2_first_half = FilteredBlockPushLowdimDataset(dataset_first_half, filter_block1_or_block2_to_region1)
filtered_dataset2_second_half = FilteredBlockPushLowdimDataset(dataset_second_half, filter_block1_or_block2_to_region1)

# Apply filter to remove trajectories where two blocks move together for dataset2
filtered_dataset2_first_half = FilteredBlockPushLowdimDataset(filtered_dataset2_first_half, filter_no_two_blocks_moving_together)
filtered_dataset2_second_half = FilteredBlockPushLowdimDataset(filtered_dataset2_second_half, filter_no_two_blocks_moving_together)

# Combine filtered datasets to create dataset2
dataset2 = torch.utils.data.ConcatDataset([filtered_dataset2_first_half, filtered_dataset2_second_half])
print("Filtered dataset2 created containing trajectories involving pushing block1 or block2 to region1.")


NameError: name 'filter_block1_to_region1_or_region2' is not defined

In [ ]:
# Visualization function for dataset1 and dataset2
def visualize_trajectory(obs, actions, save_path="trajectory.mp4"):
    fig, ax = plt.subplots()
    ax.set_xlim(-0.5, 1.5)
    ax.set_ylim(-0.5, 1.5)
    agent_patch = Rectangle((0, 0), 0.02, 0.02, fc='blue', label='Effector')
    block_patch = Rectangle((0, 0), 0.03, 0.03, fc='red', label='Block')
    block2_patch = Rectangle((0, 0), 0.03, 0.03, fc='green', label='Block 2')
    target_patch = Circle((0, 0), 0.02, fc='orange', label='Target')
    target2_patch = Circle((0, 0), 0.02, fc='purple', label='Target 2')
    region1_circle = Circle((0, 0), 0.05, fill=False, edgecolor='orange', linestyle='--', label='Region 1 Boundary')
    region2_circle = Circle((0, 0), 0.05, fill=False, edgecolor='purple', linestyle='--', label='Region 2 Boundary')
    ax.add_patch(agent_patch)
    ax.add_patch(block_patch)
    ax.add_patch(block2_patch)
    ax.add_patch(target_patch)
    ax.add_patch(target2_patch)
    ax.add_patch(region1_circle)
    ax.add_patch(region2_circle)
    plt.legend()

    def update(frame):
        block_translation = obs[frame][:2]
        block_orientation = obs[frame][2]
        block2_translation = obs[frame][3:5]
        block2_orientation = obs[frame][5]
        effector_translation = obs[frame][6:8]
        effector_target_translation = obs[frame][8:10]
        target_translation = obs[frame][10:12]
        target_orientation = obs[frame][12]
        target2_translation = obs[frame][13:15]
        target2_orientation = obs[frame][15]
        target_agent_x, target_agent_y = actions[frame][:2]
        agent_patch.set_xy((effector_translation[0], effector_translation[1]))
        block_patch.set_xy((block_translation[0], block_translation[1]))
        block2_patch.set_xy((block2_translation[0], block2_translation[1]))
        target_patch.set_center((target_translation[0], target_translation[1]))
        target2_patch.set_center((target2_translation[0], target2_translation[1]))
        region1_circle.set_center((target_translation[0], target_translation[1]))
        region2_circle.set_center((target2_translation[0], target2_translation[1]))
        ax.set_title(f"Frame: {frame}, Target Agent: ({target_agent_x:.2f}, {target_agent_y:.2f}" + "\n"
                     f"Frame: {frame}, Current Agent: ({effector_translation[0]:.2f}, {effector_target_translation[1]:.2f})")
        return agent_patch, block_patch, block2_patch, target_patch, target2_patch, region1_circle, region2_circle

    ani = animation.FuncAnimation(fig, update, frames=len(obs), blit=False, repeat=False)
    ani.save(save_path, writer='ffmpeg', fps=5)
    plt.close()


In [ ]:
import random

def visualize_dataset(dataset, save_dir, dataset_name):
    os.makedirs(save_dir, exist_ok=True)
    indices = random.sample(range(len(dataset)), min(len(dataset), 3))  # Pick 10 random trajectories for visualization
    for i in indices:  # Visualize up to 5 trajectories for verification
        data = dataset[i]
        obs = data['obs'].numpy()
        actions = data['action'].numpy()
        save_path = os.path.join(save_dir, f"{dataset_name}_trajectory_{i}.mp4")
        visualize_trajectory(obs, actions, save_path)
        print(f"{dataset_name} trajectory {i} visualization saved as {save_path}")

# Visualize and save trajectories from dataset1 and dataset2
visualize_dataset(dataset1, save_dir="visualizations/dataset1", dataset_name="dataset1")
visualize_dataset(dataset2, save_dir="visualizations/dataset2", dataset_name="dataset2")


2024-11-26 19:11:07,494 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
2024-11-26 19:11:07,496 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 640x480 -pix_fmt rgba -r 5 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -y visualizations/dataset1/dataset1_trajectory_640.mp4
2024-11-26 19:11:20,199 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
2024-11-26 19:11:20,201 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 640x480 -pix_fmt rgba -r 5 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -y visualizations/dataset1/dataset1_trajectory_688.mp4


dataset1 trajectory 640 visualization saved as visualizations/dataset1/dataset1_trajectory_640.mp4


2024-11-26 19:11:34,781 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
2024-11-26 19:11:34,783 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 640x480 -pix_fmt rgba -r 5 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -y visualizations/dataset1/dataset1_trajectory_491.mp4


dataset1 trajectory 688 visualization saved as visualizations/dataset1/dataset1_trajectory_688.mp4


2024-11-26 19:11:46,969 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
2024-11-26 19:11:46,971 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 640x480 -pix_fmt rgba -r 5 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -y visualizations/dataset2/dataset2_trajectory_34.mp4


dataset1 trajectory 491 visualization saved as visualizations/dataset1/dataset1_trajectory_491.mp4


2024-11-26 19:11:57,356 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
2024-11-26 19:11:57,358 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 640x480 -pix_fmt rgba -r 5 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -y visualizations/dataset2/dataset2_trajectory_270.mp4


dataset2 trajectory 34 visualization saved as visualizations/dataset2/dataset2_trajectory_34.mp4


2024-11-26 19:12:07,682 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
2024-11-26 19:12:07,684 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 640x480 -pix_fmt rgba -r 5 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -y visualizations/dataset2/dataset2_trajectory_1304.mp4


dataset2 trajectory 270 visualization saved as visualizations/dataset2/dataset2_trajectory_270.mp4
dataset2 trajectory 1304 visualization saved as visualizations/dataset2/dataset2_trajectory_1304.mp4


In [ ]:
def categorize_trajectories(dataset, dataset_name):
    categories = {
        "pushing_block1_to_region1": 0,
        "pushing_block2_to_region1": 0,
        "pushing_block1_to_region2": 0
    }
    pushing_block1_to_region2_indices = []  # To record indices of incorrect cases

    vs = [0,0,0]
    for idx, data in enumerate(dataset):
        obs = data['obs'].numpy()

        region1_center = obs[0][10:12]
        region2_center = obs[0][13:15]

        block1_translation_initial = obs[0][:2]
        block2_translation_initial = obs[0][3:5]
        block1_translation_final = obs[-1][:2]
        block2_translation_final = obs[-1][3:5]
        region_radius = 0.05

        # Check conditions for pushing block1 to region1
        if (np.linalg.norm(block1_translation_initial - region1_center) > region_radius and
                np.linalg.norm(block1_translation_final - region1_center) < region_radius):
            categories["pushing_block1_to_region1"] += 1

            if vs[0] == 0:
                actions = data['action'].numpy()
                save_path = os.path.join(f'./visualizations/{dataset_name}', f"{dataset_name}_trajectory_{idx}_block1_region1.mp4")
                visualize_trajectory(obs, actions, save_path)
                vs[0] = 1

        # Check conditions for pushing block2 to region1
        elif (np.linalg.norm(block2_translation_initial - region1_center) > region_radius and
                np.linalg.norm(block2_translation_final - region1_center) < region_radius):
            categories["pushing_block2_to_region1"] += 1

            if vs[1] == 0:
                actions = data['action'].numpy()
                save_path = os.path.join('./visualizations', f"{dataset_name}_trajectory_{idx}_block2_region1.mp4")
                visualize_trajectory(obs, actions, save_path)
                vs[1] = 1

        # Check conditions for pushing block1 to region2
        elif (np.linalg.norm(block1_translation_initial - region2_center) > region_radius and
                np.linalg.norm(block1_translation_final - region2_center) < region_radius):
            categories["pushing_block1_to_region2"] += 1

            if vs[2] == 0:
                actions = data['action'].numpy()
                save_path = os.path.join(f'./visualizations/{dataset_name}', f"{dataset_name}_trajectory_{idx}_block1_region2.mp4")
                visualize_trajectory(obs, actions, save_path)
                vs[2] = 1

    print("Trajectory categorization statistics:")
    for category, count in categories.items():
        print(f"{category}: {count}")

    # Print the indices of the incorrect cases for dataset2
    if pushing_block1_to_region2_indices:
        print("Indices of trajectories in dataset2 that incorrectly involve pushing block1 to region2:")
        print(pushing_block1_to_region2_indices)
        obs = data['obs'].numpy()

   

# Categorize trajectories in dataset1 and dataset2
categorize_trajectories(dataset1, 'dataset1')
categorize_trajectories(dataset2, 'dataset2')

2024-11-27 11:23:42,037 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
2024-11-27 11:23:42,039 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 640x480 -pix_fmt rgba -r 5 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -y ./visualizations/dataset1/dataset1_trajectory_0_block1_region2.mp4
2024-11-27 11:23:57,921 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
2024-11-27 11:23:57,924 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 640x480 -pix_fmt rgba -r 5 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -y ./visualizations/dataset1/dataset1_trajectory_7_block1_region1.mp4
2024-11-27 11:24:12,710 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
2024-11-27 11:24:12,713 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 640x480 -pix_fmt rgba -r 5 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -y ./visual

Trajectory categorization statistics:
pushing_block1_to_region1: 621
pushing_block2_to_region1: 0
pushing_block1_to_region2: 674


2024-11-27 11:24:26,192 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
2024-11-27 11:24:26,193 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 640x480 -pix_fmt rgba -r 5 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -y ./visualizations/dataset2_trajectory_2_block2_region1.mp4


Trajectory categorization statistics:
pushing_block1_to_region1: 723
pushing_block2_to_region1: 845
pushing_block1_to_region2: 0


In [81]:
# parameters
pred_horizon = 16
obs_horizon = 2
action_horizon = 8
#|o|o|                             observations: 2
#| |a|a|a|a|a|a|a|a|               actions executed: 8
#|p|p|p|p|p|p|p|p|p|p|p|p|p|p|p|p| actions predicted: 16

# Reload dataset1 from Zarr with new parameters
dataset1_new = BlockPushLowdimDataset(
    zarr_path="./data/training/block_pushing/dataset1.zarr",
    horizon=pred_horizon,
    pad_before=obs_horizon-1,
    pad_after=action_horizon-1
)

# Reload dataset2 from Zarr with new parameters
dataset2_new = BlockPushLowdimDataset(
    zarr_path="./data/training/block_pushing/dataset2.zarr",
    horizon=pred_horizon,
    pad_before=obs_horizon-1,
    pad_after=action_horizon-1
)


JSONDecodeError: Expecting value: line 1 column 155 (char 154)

In [6]:
#@markdown ### **Network**
#@markdown
#@markdown Defines a 1D UNet architecture `ConditionalUnet1D`
#@markdown as the noies prediction network
#@markdown
#@markdown Components
#@markdown - `SinusoidalPosEmb` Positional encoding for the diffusion iteration k
#@markdown - `Downsample1d` Strided convolution to reduce temporal resolution
#@markdown - `Upsample1d` Transposed convolution to increase temporal resolution
#@markdown - `Conv1dBlock` Conv1d --> GroupNorm --> Mish
#@markdown - `ConditionalResidualBlock1D` Takes two inputs `x` and `cond`. \
#@markdown `x` is passed through 2 `Conv1dBlock` stacked together with residual connection.
#@markdown `cond` is applied to `x` with [FiLM](https://arxiv.org/abs/1709.07871) conditioning.

class SinusoidalPosEmb(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim

    def forward(self, x):
        device = x.device
        half_dim = self.dim // 2
        emb = math.log(10000) / (half_dim - 1)
        emb = torch.exp(torch.arange(half_dim, device=device) * -emb)
        emb = x[:, None] * emb[None, :]
        emb = torch.cat((emb.sin(), emb.cos()), dim=-1)
        return emb


class Downsample1d(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.conv = nn.Conv1d(dim, dim, 3, 2, 1)

    def forward(self, x):
        return self.conv(x)

class Upsample1d(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.conv = nn.ConvTranspose1d(dim, dim, 4, 2, 1)

    def forward(self, x):
        return self.conv(x)


class Conv1dBlock(nn.Module):
    '''
        Conv1d --> GroupNorm --> Mish
    '''

    def __init__(self, inp_channels, out_channels, kernel_size, n_groups=8):
        super().__init__()

        self.block = nn.Sequential(
            nn.Conv1d(inp_channels, out_channels, kernel_size, padding=kernel_size // 2),
            nn.GroupNorm(n_groups, out_channels),
            nn.Mish(),
        )

    def forward(self, x):
        return self.block(x)


class ConditionalResidualBlock1D(nn.Module):
    def __init__(self,
            in_channels,
            out_channels,
            cond_dim,
            kernel_size=3,
            n_groups=8):
        super().__init__()

        self.blocks = nn.ModuleList([
            Conv1dBlock(in_channels, out_channels, kernel_size, n_groups=n_groups),
            Conv1dBlock(out_channels, out_channels, kernel_size, n_groups=n_groups),
        ])

        # FiLM modulation https://arxiv.org/abs/1709.07871
        # predicts per-channel scale and bias
        cond_channels = out_channels * 2
        self.out_channels = out_channels
        self.cond_encoder = nn.Sequential(
            nn.Mish(),
            nn.Linear(cond_dim, cond_channels),
            nn.Unflatten(-1, (-1, 1))
        )

        # make sure dimensions compatible
        self.residual_conv = nn.Conv1d(in_channels, out_channels, 1) \
            if in_channels != out_channels else nn.Identity()

    def forward(self, x, cond):
        '''
            x : [ batch_size x in_channels x horizon ]
            cond : [ batch_size x cond_dim]

            returns:
            out : [ batch_size x out_channels x horizon ]
        '''
        out = self.blocks[0](x)
        embed = self.cond_encoder(cond)

        embed = embed.reshape(
            embed.shape[0], 2, self.out_channels, 1)
        scale = embed[:,0,...]
        bias = embed[:,1,...]
        out = scale * out + bias

        out = self.blocks[1](out)
        out = out + self.residual_conv(x)
        return out


class ConditionalUnet1D(nn.Module):
    def __init__(self,
        input_dim,
        global_cond_dim,
        diffusion_step_embed_dim=256,
        down_dims=[256,512,1024],
        kernel_size=5,
        n_groups=8
        ):
        """
        input_dim: Dim of actions.
        global_cond_dim: Dim of global conditioning applied with FiLM
          in addition to diffusion step embedding. This is usually obs_horizon * obs_dim
        diffusion_step_embed_dim: Size of positional encoding for diffusion iteration k
        down_dims: Channel size for each UNet level.
          The length of this array determines numebr of levels.
        kernel_size: Conv kernel size
        n_groups: Number of groups for GroupNorm
        """

        super().__init__()
        all_dims = [input_dim] + list(down_dims)
        start_dim = down_dims[0]

        dsed = diffusion_step_embed_dim
        diffusion_step_encoder = nn.Sequential(
            SinusoidalPosEmb(dsed),
            nn.Linear(dsed, dsed * 4),
            nn.Mish(),
            nn.Linear(dsed * 4, dsed),
        )
        cond_dim = dsed + global_cond_dim

        in_out = list(zip(all_dims[:-1], all_dims[1:]))
        mid_dim = all_dims[-1]
        self.mid_modules = nn.ModuleList([
            ConditionalResidualBlock1D(
                mid_dim, mid_dim, cond_dim=cond_dim,
                kernel_size=kernel_size, n_groups=n_groups
            ),
            ConditionalResidualBlock1D(
                mid_dim, mid_dim, cond_dim=cond_dim,
                kernel_size=kernel_size, n_groups=n_groups
            ),
        ])

        down_modules = nn.ModuleList([])
        for ind, (dim_in, dim_out) in enumerate(in_out):
            is_last = ind >= (len(in_out) - 1)
            down_modules.append(nn.ModuleList([
                ConditionalResidualBlock1D(
                    dim_in, dim_out, cond_dim=cond_dim,
                    kernel_size=kernel_size, n_groups=n_groups),
                ConditionalResidualBlock1D(
                    dim_out, dim_out, cond_dim=cond_dim,
                    kernel_size=kernel_size, n_groups=n_groups),
                Downsample1d(dim_out) if not is_last else nn.Identity()
            ]))

        up_modules = nn.ModuleList([])
        for ind, (dim_in, dim_out) in enumerate(reversed(in_out[1:])):
            is_last = ind >= (len(in_out) - 1)
            up_modules.append(nn.ModuleList([
                ConditionalResidualBlock1D(
                    dim_out*2, dim_in, cond_dim=cond_dim,
                    kernel_size=kernel_size, n_groups=n_groups),
                ConditionalResidualBlock1D(
                    dim_in, dim_in, cond_dim=cond_dim,
                    kernel_size=kernel_size, n_groups=n_groups),
                Upsample1d(dim_in) if not is_last else nn.Identity()
            ]))

        final_conv = nn.Sequential(
            Conv1dBlock(start_dim, start_dim, kernel_size=kernel_size),
            nn.Conv1d(start_dim, input_dim, 1),
        )

        self.diffusion_step_encoder = diffusion_step_encoder
        self.up_modules = up_modules
        self.down_modules = down_modules
        self.final_conv = final_conv

        print("number of parameters: {:e}".format(
            sum(p.numel() for p in self.parameters()))
        )

    def forward(self,
            sample: torch.Tensor,
            timestep: Union[torch.Tensor, float, int],
            global_cond=None):
        """
        x: (B,T,input_dim)
        timestep: (B,) or int, diffusion step
        global_cond: (B,global_cond_dim)
        output: (B,T,input_dim)
        """
        # (B,T,C)
        sample = sample.moveaxis(-1,-2)
        # (B,C,T)

        # 1. time
        timesteps = timestep
        if not torch.is_tensor(timesteps):
            # TODO: this requires sync between CPU and GPU. So try to pass timesteps as tensors if you can
            timesteps = torch.tensor([timesteps], dtype=torch.long, device=sample.device)
        elif torch.is_tensor(timesteps) and len(timesteps.shape) == 0:
            timesteps = timesteps[None].to(sample.device)
        # broadcast to batch dimension in a way that's compatible with ONNX/Core ML
        timesteps = timesteps.expand(sample.shape[0])

        global_feature = self.diffusion_step_encoder(timesteps)

        if global_cond is not None:
            global_feature = torch.cat([
                global_feature, global_cond
            ], axis=-1)

        x = sample
        h = []
        for idx, (resnet, resnet2, downsample) in enumerate(self.down_modules):
            x = resnet(x, global_feature)
            x = resnet2(x, global_feature)
            h.append(x)
            x = downsample(x)

        for mid_module in self.mid_modules:
            x = mid_module(x, global_feature)

        for idx, (resnet, resnet2, upsample) in enumerate(self.up_modules):
            x = torch.cat((x, h.pop()), dim=1)
            x = resnet(x, global_feature)
            x = resnet2(x, global_feature)
            x = upsample(x)

        x = self.final_conv(x)

        # (B,C,T)
        x = x.moveaxis(-1,-2)
        # (B,T,C)
        return x


In [9]:
#@markdown ### **Network Demo**

# observation and action dimensions corrsponding to
# the output of PushTEnv
# obs_dim = 5
obs_dim = 16
action_dim = 2

# create network object
noise_pred_net = ConditionalUnet1D(
    input_dim=action_dim,
    global_cond_dim=obs_dim*obs_horizon
)

# example inputs
noised_action = torch.randn((1, pred_horizon, action_dim))
obs = torch.zeros((1, obs_horizon, obs_dim))
diffusion_iter = torch.zeros((1,))

# the noise prediction network
# takes noisy action, diffusion iteration and observation as input
# predicts the noise added to action
noise = noise_pred_net(
    sample=noised_action,
    timestep=diffusion_iter,
    global_cond=obs.flatten(start_dim=1))

# illustration of removing noise
# the actual noise removal is performed by NoiseScheduler
# and is dependent on the diffusion noise schedule
denoised_action = noised_action - noise

# for this demo, we use DDPMScheduler with 100 diffusion iterations
num_diffusion_iters = 100
noise_scheduler = DDPMScheduler(
    num_train_timesteps=num_diffusion_iters,
    # the choise of beta schedule has big impact on performance
    # we found squared cosine works the best
    beta_schedule='squaredcos_cap_v2',
    # clip output to [-1,1] to improve stability
    clip_sample=True,
    # our network predicts noise (instead of denoised action)
    prediction_type='epsilon'
)

# device transfer
device = torch.device('cuda')
_ = noise_pred_net.to(device)

number of parameters: 6.566861e+07


In [13]:
#@markdown ### **Training**
#@markdown
#@markdown Takes about an hour. If you don't want to wait, skip to the next cell
#@markdown to load pre-trained weights

num_epochs = 100

# Exponential Moving Average
# accelerates training and improves stability
# holds a copy of the model weights
ema = EMAModel(
    parameters=noise_pred_net.parameters(),
    power=0.75)

# Standard ADAM optimizer
# Note that EMA parametesr are not optimized
optimizer = torch.optim.AdamW(
    params=noise_pred_net.parameters(),
    lr=1e-4, weight_decay=1e-6)

# Cosine LR schedule with linear warmup
lr_scheduler = get_scheduler(
    name='cosine',
    optimizer=optimizer,
    num_warmup_steps=500,
    num_training_steps=len(dataloader) * num_epochs
)

with tqdm(range(num_epochs), desc='Epoch') as tglobal:
    # epoch loop
    for epoch_idx in tglobal:
        epoch_loss = list()
        # batch loop
        with tqdm(dataloader, desc='Batch', leave=False) as tepoch:
            for nbatch in tepoch:
                # data normalized in dataset
                # device transfer
                nobs = nbatch['obs'].to(device)
                naction = nbatch['action'].to(device)
                B = nobs.shape[0]

                # observation as FiLM conditioning
                # (B, obs_horizon, obs_dim)
                obs_cond = nobs[:,:obs_horizon,:]
                # (B, obs_horizon * obs_dim)
                obs_cond = obs_cond.flatten(start_dim=1)

                # sample noise to add to actions
                noise = torch.randn(naction.shape, device=device)

                # sample a diffusion iteration for each data point
                timesteps = torch.randint(
                    0, noise_scheduler.config.num_train_timesteps,
                    (B,), device=device
                ).long()

                # add noise to the clean images according to the noise magnitude at each diffusion iteration
                # (this is the forward diffusion process)
                noisy_actions = noise_scheduler.add_noise(
                    naction, noise, timesteps)

                # predict the noise residual
                noise_pred = noise_pred_net(
                    noisy_actions, timesteps, global_cond=obs_cond)

                # L2 loss
                loss = nn.functional.mse_loss(noise_pred, noise)

                # optimize
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                # step lr scheduler every batch
                # this is different from standard pytorch behavior
                lr_scheduler.step()

                # update Exponential Moving Average of the model weights
                ema.step(noise_pred_net.parameters())

                # logging
                loss_cpu = loss.item()
                epoch_loss.append(loss_cpu)
                tepoch.set_postfix(loss=loss_cpu)
        tglobal.set_postfix(loss=np.mean(epoch_loss))

# Weights of the EMA model
# is used for inference
ema_noise_pred_net = noise_pred_net
ema.copy_to(ema_noise_pred_net.parameters())

Epoch:   1%|          | 1/100 [00:59<1:37:54, 59.34s/it, loss=0.0598]


KeyboardInterrupt: 

In [ ]:
# Training code for two separate models
num_epochs = 100

# Training for model 1
ema1 = EMAModel(parameters=noise_pred_net.parameters(), power=0.75)
optimizer1 = torch.optim.AdamW(params=noise_pred_net.parameters(), lr=1e-4, weight_decay=1e-6)
lr_scheduler1 = get_scheduler(
    name='cosine',
    optimizer=optimizer1,
    num_warmup_steps=500,
    num_training_steps=len(filtered_dataloader1) * num_epochs
)

with tqdm(range(num_epochs), desc='Model 1 Training Epoch') as tglobal:
    for epoch_idx in tglobal:
        epoch_loss = list()
        with tqdm(filtered_dataloader1, desc='Batch', leave=False) as tepoch:
            for nbatch in tepoch:
                nobs = nbatch['obs'].to(device)
                naction = nbatch['action'].to(device)
                B = nobs.shape[0]
                obs_cond = nobs[:, :obs_horizon, :].flatten(start_dim=1)
                noise = torch.randn(naction.shape, device=device)
                timesteps = torch.randint(0, noise_scheduler.config.num_train_timesteps, (B,), device=device).long()
                noisy_actions = noise_scheduler.add_noise(naction, noise, timesteps)
                noise_pred = noise_pred_net(noisy_actions, timesteps, global_cond=obs_cond)
                loss = nn.functional.mse_loss(noise_pred, noise)
                loss.backward()
                optimizer1.step()
                optimizer1.zero_grad()
                lr_scheduler1.step()
                ema1.step(noise_pred_net.parameters())
                loss_cpu = loss.item()
                epoch_loss.append(loss_cpu)
                tepoch.set_postfix(loss=loss_cpu)
        tglobal.set_postfix(loss=np.mean(epoch_loss))

# Training for model 2
ema2 = EMAModel(parameters=noise_pred_net.parameters(), power=0.75)
optimizer2 = torch.optim.AdamW(params=noise_pred_net.parameters(), lr=1e-4, weight_decay=1e-6)
lr_scheduler2 = get_scheduler(
    name='cosine',
    optimizer=optimizer2,
    num_warmup_steps=500,
    num_training_steps=len(filtered_dataloader2) * num_epochs
)

with tqdm(range(num_epochs), desc='Model 2 Training Epoch') as tglobal:
    for epoch_idx in tglobal:
        epoch_loss = list()
        with tqdm(filtered_dataloader2, desc='Batch', leave=False) as tepoch:
            for nbatch in tepoch:
                nobs = nbatch['obs'].to(device)
                naction = nbatch['action'].to(device)
                B = nobs.shape[0]
                obs_cond = nobs[:, :obs_horizon, :].flatten(start_dim=1)
                noise = torch.randn(naction.shape, device=device)
                timesteps = torch.randint(0, noise_scheduler.config.num_train_timesteps, (B,), device=device).long()
                noisy_actions = noise_scheduler.add_noise(naction, noise, timesteps)
                noise_pred = noise_pred_net(noisy_actions, timesteps, global_cond=obs_cond)
                loss = nn.functional.mse_loss(noise_pred, noise)
                loss.backward()
                optimizer2.step()
                optimizer2.zero_grad()
                lr_scheduler2.step()
                ema2.step(noise_pred_net.parameters())
                loss_cpu = loss.item()
                epoch_loss.append(loss_cpu)
                tepoch.set_postfix(loss=loss_cpu)
        tglobal.set_postfix(loss=np.mean(epoch_loss))

# Weights of the EMA model are used for inference
ema_noise_pred_net1 = noise_pred_net
ema1.copy_to(ema_noise_pred_net1.parameters())

ema_noise_pred_net2 = noise_pred_net
ema2.copy_to(ema_noise_pred_net2.parameters())

print("Training of both models completed.")


In [16]:
#@markdown ### **Loading Pretrained Checkpoint**
#@markdown Set `load_pretrained = True` to load pretrained weights.

load_pretrained = True
if load_pretrained:
  # ckpt_path = "pusht_state_100ep.ckpt"
  ckpt_path = "data/experiments/low_dim/block_pushing/diffusion_policy_cnn/train_0/checkpoints/latest.ckpt"
  # if not os.path.isfile(ckpt_path):
      # id = "1mHDr_DEZSdiGo9yecL50BBQYzR8Fjhl_&confirm=t"
      # gdown.download(id=id, output=ckpt_path, quiet=False)

  # state_dict = torch.load(ckpt_path, map_location='cuda')
  state_dict = torch.load(ckpt_path, map_location='cuda', weights_only=False)
  ema_noise_pred_net = noise_pred_net
  ema_noise_pred_net.load_state_dict(state_dict)
  print('Pretrained weights loaded.')
else:
  print("Skipped pretrained weight loading.")

RuntimeError: Error(s) in loading state_dict for ConditionalUnet1D:
	Missing key(s) in state_dict: "mid_modules.0.blocks.0.block.0.weight", "mid_modules.0.blocks.0.block.0.bias", "mid_modules.0.blocks.0.block.1.weight", "mid_modules.0.blocks.0.block.1.bias", "mid_modules.0.blocks.1.block.0.weight", "mid_modules.0.blocks.1.block.0.bias", "mid_modules.0.blocks.1.block.1.weight", "mid_modules.0.blocks.1.block.1.bias", "mid_modules.0.cond_encoder.1.weight", "mid_modules.0.cond_encoder.1.bias", "mid_modules.1.blocks.0.block.0.weight", "mid_modules.1.blocks.0.block.0.bias", "mid_modules.1.blocks.0.block.1.weight", "mid_modules.1.blocks.0.block.1.bias", "mid_modules.1.blocks.1.block.0.weight", "mid_modules.1.blocks.1.block.0.bias", "mid_modules.1.blocks.1.block.1.weight", "mid_modules.1.blocks.1.block.1.bias", "mid_modules.1.cond_encoder.1.weight", "mid_modules.1.cond_encoder.1.bias", "diffusion_step_encoder.1.weight", "diffusion_step_encoder.1.bias", "diffusion_step_encoder.3.weight", "diffu

In [ ]:
import dill
import hydra
from diffusion_policy.workspace.base_workspace import BaseWorkspace

# load checkpoint
ckpt_path = "data/experiments/low_dim/block_pushing/diffusion_policy_cnn/train_2/checkpoints/latest.ckpt"
output_dir = "data/trash"
payload = torch.load(open(ckpt_path, 'rb'), pickle_module=dill) 
cfg = payload['cfg']
cls = hydra.utils.get_class(cfg._target_)
workspace = cls(cfg, output_dir=output_dir)
workspace: BaseWorkspace
workspace.load_payload(payload, exclude_keys=None, include_keys=None)


# get policy from workspace
policy = workspace.model
if cfg.training.use_ema:
    policy = workspace.ema_model

device = torch.device(device)
policy.to(device)
policy.eval()


2024-11-15 15:40:30,008 [INFO] number of parameters: 6.566861e+07


DiffusionUnetLowdimPolicy(
  (model): ConditionalUnet1D(
    (mid_modules): ModuleList(
      (0-1): 2 x ConditionalResidualBlock1D(
        (blocks): ModuleList(
          (0-1): 2 x Conv1dBlock(
            (block): Sequential(
              (0): Conv1d(1024, 1024, kernel_size=(5,), stride=(1,), padding=(2,))
              (1): GroupNorm(8, 1024, eps=1e-05, affine=True)
              (2): Mish()
            )
          )
        )
        (cond_encoder): Sequential(
          (0): Mish()
          (1): Linear(in_features=288, out_features=2048, bias=True)
          (2): Rearrange('batch t -> batch t 1')
        )
        (residual_conv): Identity()
      )
    )
    (diffusion_step_encoder): Sequential(
      (0): SinusoidalPosEmb()
      (1): Linear(in_features=256, out_features=1024, bias=True)
      (2): Mish()
      (3): Linear(in_features=1024, out_features=256, bias=True)
    )
    (up_modules): ModuleList(
      (0): ModuleList(
        (0): ConditionalResidualBlock1D(
       

In [11]:
#@markdown ### **Inference**

from typing import Dict, Callable, List
# limit enviornment interaction to 200 steps before termination
max_steps = 200
# env = PushTEnv()
env = BlockPushMultimodal()
# use a seed >200 to avoid initial states seen in the training dataset
env.seed(100000)

# get first observation
obs = env.reset()

policy.reset()

# keep a queue of last 2 steps of observations
obs_deque = collections.deque(
    [obs] * obs_horizon, maxlen=obs_horizon)
# save visualization and rewards
imgs = [env.render(mode='rgb_array')]
rewards = list()
done = False
step_idx = 0


def dict_apply(
        x: Dict[str, torch.Tensor], 
        func: Callable[[torch.Tensor], torch.Tensor]
        ) -> Dict[str, torch.Tensor]:
    result = dict()
    for key, value in x.items():
        if isinstance(value, dict):
            result[key] = dict_apply(value, func)
        else:
            result[key] = func(value)
    return result

# with tqdm(total=max_steps, desc="Eval PushTStateEnv") as pbar:
with tqdm(total=max_steps, desc="Eval BlockPushLowdim") as pbar:
    while not done:
        with torch.no_grad():
            np_obs_dict = {
                'obs': obs.astype(np.float32)
            }
            # device transfer
            obs_dict = dict_apply(np_obs_dict, 
                lambda x: torch.from_numpy(x).to(
                device=device))
            action_dict = policy.predict_action(obs_dict)
        # B = 1
        # # stack the last obs_horizon (2) number of observations
        # obs_seq = np.stack(obs_deque)
        # # normalize observation
        # nobs = normalize_data(obs_seq, stats=stats['obs'])
        # # device transfer
        # nobs = torch.from_numpy(nobs).to(device, dtype=torch.float32)

        # # infer action
        # with torch.no_grad():
        #     # reshape observation to (B,obs_horizon*obs_dim)
        #     obs_cond = nobs.unsqueeze(0).flatten(start_dim=1)

        #     # initialize action from Guassian noise
        #     noisy_action = torch.randn(
        #         (B, pred_horizon, action_dim), device=device)
        #     naction = noisy_action

        #     # init scheduler
        #     noise_scheduler.set_timesteps(num_diffusion_iters)

        #     for k in noise_scheduler.timesteps:
        #         # predict noise
        #         noise_pred = ema_noise_pred_net(
        #             sample=naction,
        #             timestep=k,
        #             global_cond=obs_cond
        #         )

        #         # inverse diffusion step (remove noise)
        #         naction = noise_scheduler.step(
        #             model_output=noise_pred,
        #             timestep=k,
        #             sample=naction
        #         ).prev_sample

        # unnormalize action
        naction = naction.detach().to('cpu').numpy()
        # (B, pred_horizon, action_dim)
        naction = naction[0]
        action_pred = unnormalize_data(naction, stats=stats['action'])

        # only take action_horizon number of actions
        start = obs_horizon - 1
        end = start + action_horizon
        action = action_pred[start:end,:]
        # (action_horizon, action_dim)

        # execute action_horizon number of steps
        # without replanning
        for i in range(len(action)):
            # stepping env
            obs, reward, done, _, info = env.step(action[i])
            # save observations
            obs_deque.append(obs)
            # and reward/vis
            rewards.append(reward)
            imgs.append(env.render(mode='rgb_array'))

            # update progress bar
            step_idx += 1
            pbar.update(1)
            pbar.set_postfix(reward=reward)
            if step_idx > max_steps:
                done = True
            if done:
                break

# print out the maximum target coverage
print('Score: ', max(rewards))

# visualize
from IPython.display import Video
vwrite('vis.mp4', imgs)
Video('vis.mp4', embed=True, width=256, height=256)

/home/sfchen/miniforge3/envs/robodiff/lib/python3.9/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


argv[0]=


Eval BlockPushLowdim:   0%|          | 0/200 [00:00<?, ?it/s]


AttributeError: 'collections.OrderedDict' object has no attribute 'astype'

In [18]:
m = []
sum_n, sum_d = 0, 0
for d in [0.6, 0.4]:
    for s in [0.7, 0.3]:
        if d == 0.6 and s == 0.7:
            m = [0.7, 0.3]
        elif d == 0.6 and s == 0.3:
            m = [0.4, 0.6]
        elif d == 0.4 and s == 0.7:
            m = [0.5, 0.5]
        elif d == 0.4 and s == 0.3:
            m = [0.2, 0.8]
        
        
        for i in range(len(m)):
            if d == 0.6 and i == 0:
                r = 0.3
                a = 0.2
                p = 0.4
            elif d == 0.6 and i == 1:
                r = 0.4
                a = 0.6
                p = 0.7
            elif d == 0.4 and 1 == 0:
                r = 0.5
                a = 0.2
                p = 0.4
            elif d == 0.4 and i == 1:
                r = 0.6
                a = 0.6
                p = 0.7
            
            for c in [0.5, 0.5]:
                sum_n += d * s * m[i] * r * p * a * c

for d in [0.6, 0.4]:
    for s in [0.7, 0.3]:
        if d == 0.6 and s == 0.7:
            m = [0.7, 0.3]
        elif d == 0.6 and s == 0.3:
            m = [0.4, 0.6]
        elif d == 0.4 and s == 0.7:
            m = [0.5, 0.5]
        elif d == 0.4 and s == 0.3:
            m = [0.2, 0.8]
        
        
        for i in range(len(m)):
            if d == 0.6 and i == 0:
                r = 0.3
                a = 0.2
                p = [0.6, 0.4]
            elif d == 0.6 and i == 1:
                r = 0.4
                a = 0.6
                p = [0.3, 0.7]
            elif d == 0.4 and 1 == 0:
                r = 0.5
                a = 0.2
                p = [0.6, 0.4]
            elif d == 0.4 and i == 1:
                r = 0.6
                a = 0.6
                p = [0.3, 0.7]
            
            for j in range(len(p)):
                if j == 0:
                    c = [0.7, 0.3]
                elif j == 1:
                    c = [0.5, 0.5]
                for k in range(len(c)):
                    sum_d += d * s * m[i] * r * p[j] * a * c[k]
           

0.6679135008766803
2347 3514
0.6678998292544109
5861
2348 3515
0.6679943100995732
5863


In [2]:
from fractions import Fraction

# Initialize variables
sum_n, sum_d = Fraction(0), Fraction(0)

# Loop through d and s values
for d in [Fraction(6, 10), Fraction(4, 10)]:
    for s in [Fraction(7, 10), Fraction(3, 10)]:
        # Set m values
        if d == Fraction(6, 10) and s == Fraction(7, 10):
            m = [Fraction(7, 10), Fraction(3, 10)]
        elif d == Fraction(6, 10) and s == Fraction(3, 10):
            m = [Fraction(4, 10), Fraction(6, 10)]
        elif d == Fraction(4, 10) and s == Fraction(7, 10):
            m = [Fraction(5, 10), Fraction(5, 10)]
        elif d == Fraction(4, 10) and s == Fraction(3, 10):
            m = [Fraction(2, 10), Fraction(8, 10)]

        # Iterate over m values
        for i in range(len(m)):
            # Set r, a, p values
            if d == Fraction(6, 10) and i == 0:
                r, a, p = Fraction(3, 10), Fraction(2, 10), Fraction(4, 10)
            elif d == Fraction(6, 10) and i == 1:
                r, a, p = Fraction(4, 10), Fraction(6, 10), Fraction(7, 10)
            elif d == Fraction(4, 10) and i == 0:
                r, a, p = Fraction(5, 10), Fraction(2, 10), Fraction(4, 10)
            elif d == Fraction(4, 10) and i == 1:
                r, a, p = Fraction(6, 10), Fraction(6, 10), Fraction(7, 10)

            # Iterate over c values
            for c in [Fraction(5, 10), Fraction(5, 10)]:
                sum_n += d * s * m[i] * r * p * a * c

# Loop again for second sum (sum_d)
for d in [Fraction(6, 10), Fraction(4, 10)]:
    for s in [Fraction(7, 10), Fraction(3, 10)]:
        # Set m values
        if d == Fraction(6, 10) and s == Fraction(7, 10):
            m = [Fraction(7, 10), Fraction(3, 10)]
        elif d == Fraction(6, 10) and s == Fraction(3, 10):
            m = [Fraction(4, 10), Fraction(6, 10)]
        elif d == Fraction(4, 10) and s == Fraction(7, 10):
            m = [Fraction(5, 10), Fraction(5, 10)]
        elif d == Fraction(4, 10) and s == Fraction(3, 10):
            m = [Fraction(2, 10), Fraction(8, 10)]

        # Iterate over m values
        for i in range(len(m)):
            if d == Fraction(6, 10) and i == 0:
                r, a, p = Fraction(3, 10), Fraction(2, 10), [Fraction(6, 10), Fraction(4, 10)]
            elif d == Fraction(6, 10) and i == 1:
                r, a, p = Fraction(4, 10), Fraction(6, 10), [Fraction(3, 10), Fraction(7, 10)]
            elif d == Fraction(4, 10) and i == 0:
                r, a, p = Fraction(5, 10), Fraction(2, 10), [Fraction(6, 10), Fraction(4, 10)]
            elif d == Fraction(4, 10) and i == 1:
                r, a, p = Fraction(6, 10), Fraction(6, 10), [Fraction(3, 10), Fraction(7, 10)]

            # Iterate over p and c values
            for j in range(len(p)):
                if j == 0:
                    c = [Fraction(7, 10), Fraction(3, 10)]
                elif j == 1:
                    c = [Fraction(5, 10), Fraction(5, 10)]
                for k in range(len(c)):
                    sum_d += d * s * m[i] * r * p[j] * a * c[k]

# Output results as fractions
sum_n / sum_d


Fraction(2038, 3205)

In [3]:
3205-2038

1167

In [5]:
2038+3205

5243

In [7]:
1167+2038*2

5243

In [ ]:
 
print(sum_n / sum_d)
for i in range(1, 100000):
    if -0.0001 < i / (i+1167) - sum_n/sum_d < 0.0001:
        print(i, i + 1167)
        print(i / (i+1167))
        print(2*i + 1167)

                

In [6]:
import numpy as np
from scipy.stats import multivariate_normal

# Define parameters
x = np.array([15, 40])
mean = np.array([10, 30])
covariance = np.array([[5, 0], [0, 5]])
mean2 = np.array([18, 45])
covariance2 = np.array([[8, 0], [0, 8]])

# Calculate multivariate normal PDF
pdf_value = multivariate_normal.pdf(x, mean=mean, cov=covariance) \
    / (multivariate_normal.pdf(x, mean=mean, cov=covariance) 
       + multivariate_normal.pdf(x, mean=mean2, cov=covariance2) )
print(pdf_value)
print(1 - pdf_value)
pdf_value = multivariate_normal.pdf(x, mean=mean2, cov=covariance2) \
    / (multivariate_normal.pdf(x, mean=mean, cov=covariance) 
       + multivariate_normal.pdf(x, mean=mean2, cov=covariance2) )
print(pdf_value)
print(multivariate_normal.pdf(x, mean=mean, cov=covariance))
print(multivariate_normal.pdf(x, mean=mean2, cov=covariance2))


4.9922123630939464e-05
0.999950077876369
0.9999500778763691
1.1862305470508222e-07
0.0023760434084732374


In [17]:
6 * np.log(10000) - 2*(-11500)

23055.262042231858

In [18]:
64*64*425 + 425 + 425*255+255 + 255*80+80+80*6+6

1870821

In [22]:
4096*425

1740800